<a href="https://colab.research.google.com/github/saravanamuthu1/sentiment_analysis/blob/main/sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [55]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from collections import Counter
import math
import re


In [61]:
class Naive_bayes:
  def __init__(self) -> None:
    self.mylist=[]
    self.li=[]
    self.V_freq = {}
    self.freq_pos={}
    self.freq_neg={}
    self.df=None
    self.total_porobaility_for_pos=0
    self.total_porobaility_for_neg=0
    self.liklywood_pos={}
    self.liklywood_neg={}
  def get_data_set(self,my_data):
    for line in my_data:
      value=line.rstrip("\n").split("\t")
      self.mylist.append(line.rstrip("\n").split("\t"))
    df1=pd.DataFrame(self.mylist)
    df1.columns=["label","tweet"]
    return df1,self.mylist
  #fit function
  def fit(self,df1):
    df1['train_value'] = df1.train_value.apply(lambda x:re.sub(r'[^\w\s]', '', x))
    df1['train_value'] = df1.train_value.apply(lambda x:x.strip())
    df1['train_value'] =df1.train_value.apply(lambda x:x.replace("USER", ""))
    df1["train_value"]= df1.train_value.apply(lambda x:" ".join(x.split()))
    df1["train_value"]= df1.train_value.apply(lambda x:''.join([i for i in x if not i.isdigit()]))
    self.freq_pos= Counter(" ".join(df1.loc[df1.train_label=='1']['train_value'].values.tolist()).split(" "))
    self.V_freq=Counter(" ".join(df1['train_value'].values.tolist()).split(" "))
    self.freq_neg = Counter(" ".join(df1.loc[df1.train_label=='0']['train_value'].values.tolist()).split(" ")) 
    self.total_porobaility_for_pos=np.log(len(self.freq_pos)-len(self.freq_neg))
    self.total_porobaility_for_neg=np.log(len(self.freq_pos)-len(self.freq_neg))
    for w in self.V_freq:
      if w in self.freq_pos:
        p_w_pos_value=float((1+(self.freq_pos.get(w,0)))/(len(self.V_freq)+len(self.freq_pos)))
      if w in self.freq_neg:
        p_w_neg_value = float((1+(self.freq_neg.get(w,0)))/(len(self.V_freq)+len(self.freq_neg)))
      self.liklywood_pos[w] = p_w_pos_value
      self.liklywood_neg[w] = p_w_neg_value

  # data pre-processing
  def pre_processing(self,mytest_file):
    mytest_file['train_value'] = mytest_file.train_value.apply(lambda x:re.sub(r'[^\w\s]', '', x))
    mytest_file['train_value'] = mytest_file.train_value.apply(lambda x:x.strip())
    mytest_file['train_value'] = mytest_file.train_value.apply(lambda x:x.replace("USER", ""))
    mytest_file["train_value"]= mytest_file.train_value.apply(lambda x:" ".join(x.split()))
    mytest_file["train_value"]= mytest_file.train_value.apply(lambda x:''.join([i for i in x if not i.isdigit()]))
    return mytest_file
  # predict fucntion
  def predict(self,mytxt):
    p1=0.5
    p2=0.5
    mytxt=mytxt.split()
    mytxt=list(set(mytxt))
    pos_score = self.total_porobaility_for_pos
    neg_score = self.total_porobaility_for_neg
    p=self.total_porobaility_for_pos
    for w in mytxt:
      if w in self.liklywood_pos:
        p1+=self.liklywood_pos[w]
      if w in self.liklywood_neg:
        p2+=self.liklywood_neg[w]
    total_value=max(p1,p2)
    if total_value == p1:
      return 1
    else:
      return 0

if __name__ == "__main__":
  #to fit the data
  testfile = open('train-v2.tsv', 'r')
  data_value=[]

  obj1= Naive_bayes()
  df1,total_data=obj1.get_data_set(testfile)
  # predict values
  # upload your dataset in the colab and change the name here to predict
  predict_file= open('train-v2.tsv', 'r')
  #since we don't have test set we have used split the entire corpus. we can alter still alter the program for jsut test set alone.
  train_set=total_data[0:80000]
  df2=pd.DataFrame(train_set)
  df2.columns=["train_label","train_value"]
  df2=obj1.pre_processing(df2)
  obj1.fit(df2)
  df2['train_label'] = df2.train_label.apply(lambda x:int(x))
  testfile1 = open('test.tsv', 'r')
  for line in testfile1:
      value1=line.rstrip("\n").split("\t")
      data_value.append(value1)
  df3 =pd.DataFrame(data_value)
  df3.columns=["test_label","test_value"]
  df3['test_label'] = df3.test_label.apply(lambda x:int(x))
  df2['total_prediction_train']=df2.train_value.apply(lambda x:obj1.predict(x))
  df3['total_prediction_test']=df3.test_value.apply(lambda x:obj1.predict(x))
  accuracy1 = accuracy_score(df2['train_label'], df2['total_prediction_train'])
  accuracy2 = accuracy_score(df3['test_label'], df3['total_prediction_test'])
  print("accuracy on the trianing_set:",accuracy1)
  print("accuracy on the test_set:",accuracy2)




  

    


  

accuracy on the trianing_set: 0.600825
accuracy on the test_set: 0.5991
